### 1.计算权重参数和偏置的数量

![Image Name](https://cdn.kesci.com/upload/image/q5ho684jmh.png)

给定一个小批量样本$\boldsymbol{X} \in \mathbb{R}^{n \times d}$，其批量大小为$n$，输入个数为$d$，输出个数为$q$。假设多层感知机只有一个隐藏层，其中隐藏单元个数为$h$。记隐藏层的输出（也称为隐藏层变量或隐藏变量）为$\boldsymbol{H}$，有$\boldsymbol{H} \in \mathbb{R}^{n \times h}$。因为隐藏层和输出层均是全连接层，则隐藏层的权重参数和偏差参数分别为$\boldsymbol{W}_h \in \mathbb{R}^{d \times h}$和 $\boldsymbol{b}_h \in \mathbb{R}^{1 \times h}$，输出层的权重和偏差参数分别为$\boldsymbol{W}_o \in \mathbb{R}^{h \times q}$和$\boldsymbol{b}_o \in \mathbb{R}^{1 \times q}$。

`参数的形状与批量大小无关`

例：对于只含有一个隐藏层的多层感知机，输入是256×256的图片，隐藏单元个数是1000，输出类别个数是10，求模型的所有权重矩阵$W_{i}$的元素数量之和。

解：图片256×256展平为65536，65536×1000+1000×10=65546000

### 2.广播机制

broadcast机制：广播仅仅是一组用于在不同大小的数组上应用二元ufuncs(加法、减法、乘法等)的规则。

* 规则1:如果两个数组在维度(dim=0标量, dim=1向量, dim=2矩阵)的数量上有差异，那么维度较少的数组的形状就会被用1填充在它的前导(左)边。
* 规则2:如果两个数组的形状在任何维度上都不匹配，但某个维度等于1，那么在这个维度中，形状为1的数组将被拉伸以匹配另一个形状。
* 规则3:如果在任何维度上，大小都不一致，且两者都不等于1，就会出现错误。

In [1]:
import numpy as np

M = np.ones((2, 3))
print(M)
print(M.shape)
# M.shape -> (2, 3)

a = np.arange(3)
print(a)
print(a.shape)
# a.shape -> (3, )

#根据规则1，数组a的维数更少，所以用1填充在它的左边，
# a.shape -> (1, 3)

#根据规则2，可将a变为 a.shape -> (2, 3)
x = (M + a).shape
print(x)

[[1. 1. 1.]
 [1. 1. 1.]]
(2, 3)
[0 1 2]
(3,)
(2, 3)


### 3.交叉熵

交叉熵（cross entropy）是一个常用的衡量两个概率分布差异的测量函数：


$$
H\left(\boldsymbol y^{(i)}, \boldsymbol {\hat y}^{(i)}\right ) = -\sum_{j=1}^q y_j^{(i)} \log \hat y_j^{(i)},
$$


其中带下标的$y_j^{(i)}$是向量$\boldsymbol y^{(i)}$中非0即1的元素，需要注意将它与样本$i$类别的离散数值，即不带下标的$y^{(i)}$区分。在上式中，我们知道向量$\boldsymbol y^{(i)}$中只有第$y^{(i)}$个元素$y^{(i)}{y^{(i)}}$为1，其余全为0，于是$H(\boldsymbol y^{(i)}, \boldsymbol {\hat y}^{(i)}) = -\log \hat y_{y^{(i)}}^{(i)}$。也就是说，交叉熵只关心对正确类别的预测概率，因为只要其值足够大，就可以确保分类结果正确。当然，遇到一个样本有多个标签时，例如图像里含有不止一个物体时，我们并不能做这一步简化。但即便对于这种情况，交叉熵同样只关心对图像中出现的物体类别的预测概率。

假设训练数据集的样本数为$n$，交叉熵损失函数定义为 
$$
\ell(\boldsymbol{\Theta}) = \frac{1}{n} \sum_{i=1}^n H\left(\boldsymbol y^{(i)}, \boldsymbol {\hat y}^{(i)}\right ),
$$


其中$\boldsymbol{\Theta}$代表模型参数。同样地，如果每个样本只有一个标签，那么交叉熵损失可以简写成$\ell(\boldsymbol{\Theta}) = -(1/n) \sum_{i=1}^n \log \hat y_{y^{(i)}}^{(i)}$。从另一个角度来看，我们知道最小化$\ell(\boldsymbol{\Theta})$等价于最大化$\exp(-n\ell(\boldsymbol{\Theta}))=\prod_{i=1}^n \hat y_{y^{(i)}}^{(i)}$，即最小化交叉熵损失函数等价于最大化训练数据集所有标签类别的联合预测概率。


### 4.多维Tensor按维度操作

In [ ]:
import torch

X = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(X)
print(X.sum(dim=0, keepdim=True))  # dim为0，按照相同的列求和，并在结果中保留列特征
print(X.sum(dim=1, keepdim=True))  # dim为1，按照相同的行求和，并在结果中保留行特征
print(X.sum(dim=0, keepdim=False)) # dim为0，按照相同的列求和，不在结果中保留列特征
print(X.sum(dim=1, keepdim=False)) # dim为1，按照相同的行求和，不在结果中保留行特征

'''
tensor([[1, 2, 3],
        [4, 5, 6]])

tensor([[5, 7, 9]])
tensor([[ 6],
        [15]])
tensor([5, 7, 9])
tensor([ 6, 15])
'''

### 5.几个重要的函数

In [ ]:
# view()

import torch

X = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(X)

print(X.view(-1, 1))
print(X.view(1, -1))
print(X.view(-1)) #变为一维

'''
tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[1],
        [2],
        [3],
        [4],
        [5],
        [6]])
tensor([[1, 2, 3, 4, 5, 6]])
tensor([1, 2, 3, 4, 5, 6])
'''

In [ ]:
# gather()

torch.gather(input, dim, index, out=None) → Tensor(其形状与index一致)

#例1：
b = torch.Tensor([[1,2,3],[4,5,6]])
print(b)
index_1 = torch.LongTensor([[0,1],[2,0]])
index_2 = torch.LongTensor([[0,1,1],[0,0,0]])
print(torch.gather(b, dim=1, index=index_1)) #二维时，1表示行
print(torch.gather(b, dim=0, index=index_2)) #二维时，0表示列

'''
tensor([[1., 2., 3.],
        [4., 5., 6.]])
tensor([[1., 2.],
        [6., 4.]])
tensor([[1., 5., 6.],
        [1., 2., 3.]])
'''

#例2：
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y = torch.LongTensor([0, 2])
y_hat = torch.gather(y_hat, 1, y.view(-1, 1))  #等价于y_hat.gather(1, y.view(-1, 1))
print(y_hat)

'''
tensor([[0.1000],
        [0.5000]])
'''

#例3：
a = torch.randint(0, 30, (2, 3, 5))
print(a)
'''
tensor([[[27, 25, 10,  9, 29],
         [29, 22, 19, 23, 16],
         [19,  9,  0, 25,  3]],

        [[11, 28, 26, 25, 24],
         [ 3, 14, 11,  3,  6],
         [29,  4,  0, 18,  4]]])
'''

index = torch.LongTensor([[[0,1,2,0,2],
                          [0,0,0,0,0],
                          [1,1,1,1,1]],
                        [[1,2,2,2,2],
                         [0,0,0,0,0],
                         [2,2,2,2,2]]])
print(a.size()==index.size())
b = torch.gather(a, 1, index) #三维时，1表示列
print(b)
'''
tensor([[[27, 22,  0,  9,  3],
         [27, 25, 10,  9, 29],
         [29, 22, 19, 23, 16]],

        [[ 3,  4,  0, 18,  4],
         [11, 28, 26, 25, 24],
         [29,  4,  0, 18,  4]]])
'''

c = torch.gather(a, 2, index) #三维时，2表示行
print(c)
'''
tensor([[[27, 25, 10, 27, 10],
         [29, 29, 29, 29, 29],
         [ 9,  9,  9,  9,  9]],

        [[28, 26, 26, 26, 26],
         [ 3,  3,  3,  3,  3],
         [ 0,  0,  0,  0,  0]]])
'''

index2 = torch.LongTensor([[[0,1,1,0,1],
                          [0,1,1,1,1],
                          [1,1,1,1,1]],
                        [[1,0,0,0,0],
                         [0,0,0,0,0],
                         [1,1,0,0,0]]])

d = torch.gather(a, 0, index2) #三维时，0表示分别从index对应的(n, c, l)的n个不同组元素中选取相应位置元素
print(d)
'''
例如：这里n=2，即一共有2个(3,5)列的数据，[0,1,1,0,1]分别从第一组数据[27,25,10,9,29]
中选择第0个位置的数(27),再从第二组数据[11, 28, 26, 25, 24]中选择第1个位置的数(28)，
再从第二组数据中选择第3个位置的数(26),再从第一组数据中选择第4个位置的数(9)，最后从
第二组数据中选择第5个位置的数(24)
a = tensor([[[27, 25, 10,  9, 29],
         [29, 22, 19, 23, 16],
         [19,  9,  0, 25,  3]],

        [[11, 28, 26, 25, 24],
         [ 3, 14, 11,  3,  6],
         [29,  4,  0, 18,  4]]])
         
tensor([[[27, 28, 26,  9, 24],
         [29, 14, 11,  3,  6],
         [29,  4,  0, 18,  4]],

        [[11, 25, 10,  9, 29],
         [29, 22, 19, 23, 16],
         [29,  4,  0, 25,  3]]])
'''

### 6.激活函数

#### ReLU (rectified linear unit)

ReLU函数提供了一个简单的非线性变换。给定元素$x$，该函数定义为

$$
\text{ReLU}(x) = \max(x, 0).
$$

可以看出，ReLU函数只保留正数元素，并将负数元素清零。

In [ ]:
import matplotlib.pyplot as plt

def xyplot(x_vals, y_vals, name):
    plt.plot(x_vals.detach().numpy(), y_vals.detach().numpy())
    plt.xlabel('x')
    plt.ylabel(name + '(x)')
    
x = torch.arange(-8.0, 8.0, 0.1, requires_grad=True)
y = x.relu()
xyplot(x, y, 'relu')

y.sum().backward()
xyplot(x, x.grad, 'grad of relu')

#### Sigmoid

sigmoid函数可以将元素的值变换到0和1之间：

$$
\text{sigmoid}(x) = \frac{1}{1 + \exp(-x)}.
$$

In [ ]:
y = x.sigmoid()
xyplot(x, y, 'sigmoid')

x.grad.zero_()
y.sum().backward()
xyplot(x, x.grad, 'grad of sigmoid')

#### tanh

tanh（双曲正切）函数可以将元素的值变换到-1和1之间，函数关于原点对称：


$$
\text{tanh}(x) = \frac{1 - \exp(-2x)}{1 + \exp(-2x)}.
$$

In [ ]:
y = x.tanh()
xyplot(x, y, 'tanh')

x.grad.zero_()
y.sum().backward()
xyplot(x, x.grad, 'grad of tanh')

#### 关于激活函数的选择

* ReLu函数是一个通用的激活函数，目前在大多数情况下使用。但是，`ReLU函数只能在隐藏层中使用`。
* 用于分类器时，sigmoid函数及其组合通常效果更好。由于梯度消失问题，有时要避免使用sigmoid和tanh函数。  
* 在神经网络层数较多的时候，最好使用ReLu函数，ReLu函数比较简单计算量少，而sigmoid和tanh函数计算量大很多。
* 在选择激活函数的时候可以先选用ReLu函数如果效果不理想可以尝试其他激活函数。